In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from google.colab import drive
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from tqdm.auto import tqdm
import gc
import keras
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.layers import BatchNormalization,Dropout,Conv1D,Activation,Add,Flatten,Dense
from keras.layers import MaxPooling1D
from keras.layers.merge import concatenate
from keras.initializers import glorot_uniform
from keras.layers import ZeroPadding1D
from keras.models import Input,Model
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight



import warnings 
warnings.filterwarnings('ignore')


drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('pfam_176.csv')
df['sequence'] = df['sequence'].apply(lambda x: x.upper())

In [15]:
df_train = df[df['set'] == 'train']
df_dev = df[df['set'] == 'dev']
df_test = df[df['set'] == 'test']

In [4]:
!mkdir prottrans # root directory for storing checkpoints, results etc
!mkdir prottrans/prottrans_checkpoint # directory holding the ProtT5 checkpoint
!mkdir prottrans/output # directory for storing your embeddings

In [5]:
from scipy import stats

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight

!pip3 install tensorflow transformers sentencepiece h5py

from transformers import T5EncoderModel, T5Tokenizer
import torch
import h5py

import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 6.3 MB/s 
     |████████████████████████████████| 1.3 MB 47.6 MB/s 
     |████████████████████████████████| 6.6 MB 53.5 MB/s 
     |████████████████████████████████| 120 kB 48.9 MB/s 


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [6]:
# whether to retrieve embeddings for each residue in a protein 
# --> Lx1024 matrix per protein with L being the protein's length
# as a rule of thumb: 1k proteins require around 1GB RAM/disk
per_residue = False 
per_residue_path = "./prottrans/output/per_residue_embeddings.h5" # where to store the embeddings

# whether to retrieve per-protein embeddings 
# --> only one 1024-d vector per protein, irrespective of its length
per_protein = True
per_protein_path = "./prottrans/output/per_protein_embeddings.h5" # where to store the embeddings

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [8]:
def get_T5_model():
    model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc")
    model = model.to(device) # move model to GPU
    model = model.eval() # set model to evaluation model
    tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False)

    return model, tokenizer

In [9]:
#@title Generate embeddings. { display-mode: "form" }
# Generate embeddings via batch-processing
# per_residue indicates that embeddings for each residue in a protein should be returned.
# per_protein indicates that embeddings for a whole protein should be returned (average-pooling)
# max_residues gives the upper limit of residues within one batch
# max_seq_len gives the upper sequences length for applying batch-processing
# max_batch gives the upper number of sequences per batch


def get_embeddings( model, tokenizer, seqs, per_residue, per_protein,
                   max_residues=400, max_seq_len=100, max_batch=100 ):
    results = {"residue_embs" : dict(), 
               "protein_embs" : dict()}

    # sort sequences according to length (reduces unnecessary padding --> speeds up embedding)
    seq_dict   = sorted( seqs.items(), key=lambda kv: len( seqs[kv[0]] ), reverse=True )
    start = time.time()
    batch = list()
    for seq_idx, (pdb_id, seq) in enumerate(seq_dict,1):
        seq = seq
        seq_len = len(seq)
        seq = ' '.join(list(seq))
        batch.append((pdb_id,seq,seq_len))

        # count residues in current batch and add the last sequence length to
        # avoid that batches with (n_res_batch > max_residues) get processed 
        n_res_batch = sum([ s_len for  _, _, s_len in batch ]) + seq_len 
        if len(batch) >= max_batch or n_res_batch>=max_residues or seq_idx==len(seq_dict) or seq_len>max_seq_len:
            pdb_ids, seqs, seq_lens = zip(*batch)
            batch = list()

            # add_special_tokens adds extra token at the end of each sequence
            token_encoding = tokenizer.batch_encode_plus(seqs, add_special_tokens=True, padding="longest")
            input_ids      = torch.tensor(token_encoding['input_ids']).to(device)
            attention_mask = torch.tensor(token_encoding['attention_mask']).to(device)
            
            try:
                with torch.no_grad():
                    # returns: ( batch-size x max_seq_len_in_minibatch x embedding_dim )
                    embedding_repr = model(input_ids, attention_mask=attention_mask)
            except RuntimeError:
                print("RuntimeError during embedding for {} (L={})".format(pdb_id, seq_len))
                continue

            for batch_idx, identifier in enumerate(pdb_ids): # for each protein in the current mini-batch
                s_len = seq_lens[batch_idx]
                # slice off padding --> batch-size x seq_len x embedding_dim  
                emb = embedding_repr.last_hidden_state[batch_idx,:s_len]
                if per_residue: # store per-residue embeddings (Lx1024)
                    results["residue_embs"][ identifier ] = emb.detach().cpu().numpy().squeeze()
                if per_protein: # apply average-pooling to derive per-protein embeddings (1024-d)
                    protein_emb = emb.mean(dim=0)
                    results["protein_embs"][identifier] = protein_emb.detach().cpu().numpy().squeeze()


    passed_time=time.time()-start
    avg_time = passed_time/len(results["residue_embs"]) if per_residue else passed_time/len(results["protein_embs"])
    print('\n############# EMBEDDING STATS #############')
    print('Total number of per-residue embeddings: {}'.format(len(results["residue_embs"])))
    print('Total number of per-protein embeddings: {}'.format(len(results["protein_embs"])))
    print("Time for generating embeddings: {:.1f}[m] ({:.3f}[s/protein])".format(
        passed_time/60, avg_time ))
    print('\n############# END #############')
    return results

In [10]:
def save_embeddings(emb_dict,out_path):
    with h5py.File(str(out_path), "w") as hf:
        for sequence_id, embedding in emb_dict.items():
            # noinspection PyUnboundLocalVariable
            hf.create_dataset(sequence_id, data=embedding)
    return None

In [16]:
seq_dict_train = dict(zip(df_train.sequence_name, df_train.sequence))
seq_dict_valid = dict(zip(df_dev.sequence_name, df_dev.sequence))
seq_dict_test = dict(zip(df_test.sequence_name, df_test.sequence))

## Get the embeddings

In [17]:
model, tokenizer = get_T5_model()

start = time.time()
Embedding_train = get_embeddings(model, tokenizer, seq_dict_train, per_residue, per_protein)
Embedding_valid = get_embeddings(model, tokenizer, seq_dict_valid, per_residue, per_protein)
Embedding_test = get_embeddings(model, tokenizer, seq_dict_test, per_residue, per_protein)
end = time.time()
print('Execution time: {} seconds'.format(end - start))

#Free some memory
#gc.collect()
#Embedding_train["residue_embs"] = None
#Embedding_valid["residue_embs"] = None
#Embedding_test["residue_embs"] = None

#Re-order the embeddings
#Embedding_train["residue_embs"] = {x: Embedding_train["residue_embs"][x] for x in train_df.sequence_name.to_list()}
Embedding_train["protein_embs"] = {x: Embedding_train["protein_embs"][x] for x in df_train.sequence.to_list()}

#Embedding_valid["residue_embs"] = {x: Embedding_valid["residue_embs"][x] for x in valid_df.sequence_name.to_list()}
Embedding_valid["protein_embs"] = {x: Embedding_valid["protein_embs"][x] for x in df_dev.sequence.to_list()}

#Embedding_test["residue_embs"] = {x: Embedding_test["residue_embs"][x] for x in test_df.sequence_name.to_list()}
Embedding_test["protein_embs"] = {x: Embedding_test["protein_embs"][x] for x in df_test.sequence.to_list()}

# Save the embeddings
"""
if per_residue:
  save_embeddings(Embedding_train["residue_embs"], per_residue_path.replace('.h5', '_train.h5'))
  save_embeddings(Embedding_valid["residue_embs"], per_residue_path.replace('.h5', '_valid.h5'))
  save_embeddings(Embedding_test["residue_embs"], per_residue_path.replace('.h5', '_test.h5'))
"""
if per_protein:
  save_embeddings(Embedding_train["protein_embs"], per_protein_path.replace('.h5', '_train.h5'))
  save_embeddings(Embedding_valid["protein_embs"], per_protein_path.replace('.h5', '_valid.h5'))
  save_embeddings(Embedding_test["protein_embs"], per_protein_path.replace('.h5', '_test.h5'))

Downloading:   0%|          | 0.00/656 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/238k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

KeyboardInterrupt: ignored